## Vision and Visualization (EC60338)
**Assignment - 1 (dated 02.04.2023)** \
**Using the 3D Reconstruction Algorithm (covered in class) to estimate world points in presence of noise (during capturing/recording of image points):** \
Name: Shaswata Dutta \
Roll: 19EC39034

Importing necessary libraries

In [58]:
import numpy as np
import random
import pandas as pd
import os
from os import path
import matplotlib.pyplot as plt
import scipy
from scipy import optimize
import math
import copy

**Creating the camera matrices:**

The entries are from the Standard Normal Distribution $(\mathcal{N}(0, 1))$.

In [45]:
P1 = np.random.randn(3, 4)
P2 = np.random.randn(3, 4)
print("P: ", P1, '\n')
print("P': ", P2)

P:  [[-1.08639422  0.68562776 -0.3632688  -0.17075507]
 [ 0.02009941  0.34985729  0.21712834 -0.48409077]
 [ 0.30361013 -0.57613321  1.95046921  0.57282085]] 

P':  [[ 1.07046321  2.64633248 -0.51535217  0.66428143]
 [-2.14084448  0.1422274  -0.62359856  0.80343656]
 [ 1.80562629 -0.13865285 -0.36128838 -0.0078673 ]]


**Creating an ensemble of 100 world points.** \
We store them in homogeneous form.

In [46]:
# Constructing an ensemble of 100 world points, X (in P3 space)
# Each point X_i can be accessed using X[:, i]
# X = np.zeros((4, 100))
X = np.random.randn(4, 100)

for j in range(100):
  X[3][j] = 1
print("Ensemble of world points X: ", '\n')
for i in range(4):
  for j in range(100):
    print(X[i][j], end = '\t')
  print('\n')


Ensemble of world points X:  

-0.170073485186932	0.3793897829643355	-1.0051978366891274	1.6547040147659962	0.639801825435495	1.9646615110887728	-2.354583356812444	0.6821542333476274	0.3081076521307233	-0.6919701573551995	-0.15322109802143882	1.7371446797772618	0.3415411860392536	-0.6690552790364349	0.9895496587236037	0.8822147056862009	-0.9772264451028749	-0.7540750787618918	-0.1198455931063308	-0.057165526122566535	-0.46009799560025466	0.18710593406960244	1.858558608168176	-0.23879168438112464	-0.8355427901184642	1.7930410350005448	0.5988651530680712	-1.1075367967685945	2.3869643250353794	-0.5979387893697999	0.03915477099805152	0.9588333022603754	0.19540386902964357	-0.19323432752215913	-0.32893044197688137	-0.5701193962806655	-0.19461403343179393	0.31180973879733237	0.5992153517432406	-0.3586441287362738	-0.6022457904944243	0.8577702629156285	1.9419261964649328	-1.8662699868729002	0.24217327078841497	1.3538199886263507	-1.0196427927039777	1.0927718627140768	-0.07327758280417156	0.32

The following cell is for constructing the image points from the world points above, by applying the camera matrices $P$ and $P'$ on these world points and obtaining $x = PX$, and $x' = P'X$. 

In [47]:
# Constructing image points based on the obtained world points
# x1 and x2 are the ensemble of the image points generated
x1 = np.zeros((3, 100))
x2 = np.zeros((3, 100))
for i in range(100):
  Xt = np.zeros((4, 1))
  Xt[0][0] = X[0][i]
  Xt[1][0] = X[1][i]
  Xt[2][0] = X[2][i]
  Xt[3][0] = X[3][i]
  
  tmp1 = np.matmul(P1, Xt)
  tmp2 = np.matmul(P2, Xt)
  
  tmp1 = tmp1/tmp1[2][0]
  tmp2 = tmp2/tmp2[2][0]
  x1[0, i] = tmp1[0][0]
  x1[1, i] = tmp1[1][0]
  x1[2, i] = tmp1[2][0]
  x2[0, i] = tmp2[0][0]
  x2[1, i] = tmp2[1][0]
  x2[2, i] = tmp2[2][0]

print("Ensemble of image points corresponding to camera 1 (x1): ")
for i in range(3):
  for j in range(100):
    print(x1[i][j], end = '\t')
  print('\n')
print("Ensemble of image points corresponding to camera 2 (x2): ")
for i in range(3):
  for j in range(100):
    print(x2[i][j], end = '\t')
  print('\n')

Ensemble of image points corresponding to camera 1 (x1): 
-0.2538470244312073	-0.6767361829087326	-0.761654206458475	-0.8402629064394712	-25.33323008015595	0.6942677253561913	0.9690117560856876	-1.106507122559669	0.13115883875072817	-0.44273302440548135	-11.30127346430078	-1.3214918572513916	-1.3717023100596581	-1.4043722905959433	-0.2510162925813694	0.06011419689866625	-1.002802989052537	-0.927952330483107	-0.44130492661151555	-0.42140402534890353	-0.7163531697973884	-0.7405288461365772	-0.4156163333823667	2.4628875583218344	-2.797146589220718	-1.4414096210774119	0.0466629122053838	0.48030148339600404	-2.8407742816225388	-0.2189185838716385	-0.606609818511823	-1.0201332432268202	-0.22514825119622514	-0.23641390338971824	-1.235798596588646	-0.16748798773048157	-0.1430547167874663	-0.47285557176382004	-0.1212570747054828	-0.2657730774002667	-0.11255829236122587	-0.89590818293627	-2.5676327638912286	13.913828842259694	-0.03360649611903082	-0.7429168168627033	-1.7341676294546131	0.4104243

**Choosing $k$ random points for estimating matrix F:** \
Here, we have taken $k = 20$.

In [48]:
kp = 20    # the value of k

In [49]:
arr = np.array([i for i in range(100)])
randkp = np.unique(np.random.choice(arr, size=kp, replace=False))
# randkp = np.array([jk for jk in range(20)])
print("The set of", str(kp), "random indices to build the fundamental matrix F is: ", randkp)

The set of 20 random indices to build the fundamental matrix F is:  [ 6 11 15 23 27 30 38 45 46 55 57 62 69 74 78 87 91 95 96 97]


In [50]:
mat = np.zeros((kp, 9))
for row in range(kp):
  idx = randkp[row]
  p1 = np.zeros((3, 1))
  p2 = np.zeros((3, 1))
  for j in range(3):
    p1[j] = x1[j][idx]
    p2[j] = x2[j][idx]
  mat[row][0] = p1[0]*p2[0]
  mat[row][1] = p1[1]*p2[0]
  mat[row][2] = p1[2]*p2[0]
  mat[row][3] = p1[0]*p2[1]
  mat[row][4] = p1[1]*p2[1]
  mat[row][5] = p1[2]*p2[1]
  mat[row][6] = p1[0]*p2[2]
  mat[row][7] = p1[1]*p2[2]
  mat[row][8] = p1[2]*p2[2]


We are using the **Direct Linear Transformation (DLT)**, via **SVD (Singular Value Decomposition)** technique. This technique has been used to solve equations of the form $Ax = 0$, particularly when we require non-trivial solutions, and also when $A$ is a low-rank matrix. 

In [51]:
## The following is the definition of the DLT function for extracting the last column of y
## This last column is reshaped into (3, 3) to give the estimated F matrix

def dlt(a):                     # function which performs the operation in Direct Linear Transformation Algorithm
  u, s, vh = np.linalg.svd(a, full_matrices=True, hermitian=False)
  v = np.transpose(vh)
  return v

Estimating fundamental matrix $F$ from 20 randomly chosen image points. We have used **SVD** technique.

In [52]:
# computation of F_estimated (written simply as F) 
tempF = dlt(mat)
F = tempF[:, -1].reshape((3, 3))
print(F)

[[-0.07431392  0.12740571 -0.0024944 ]
 [ 0.46550291 -0.09525132  0.1354272 ]
 [ 0.56300862 -0.64161419  0.07406233]]


Taking the 80 remaining points from the 100 total points (in both cameras 1 and 2). 

In [53]:
## Now we need to add noise to the remaining 80 points
rem80 = list(set(arr) - set(randkp))

The following cell is for **adding noise to the remaining 80 points.** We have added noise of the form $\mathcal{N}(0, \sigma^{2})$, where $\sigma$ is the standard deviation of the noise, which has $0$ mean. The value of $\sigma$ has been tweaked and the performance of the 3D reconstruction algorithm has been recorded.

In [54]:
## tempx1 and tempx2 are the ensembles of 100 image points  
## to which we will add noise
## sigma (standard deviation) of noise = 0.1 (can be tweaked)

# mean and standard deviation
mu_n = 0
sigma_n = 0.1

tempx1 = np.zeros((3, 100))
tempx2 = np.zeros((3, 100))
for i in rem80:
  tempx1[0][i] = x1[0][i] + sigma_n*np.random.randn()
  tempx1[1][i] = x1[1][i] + sigma_n*np.random.randn()
  tempx2[0][i] = x2[0][i] + sigma_n*np.random.randn()
  tempx2[1][i] = x2[1][i] + sigma_n*np.random.randn()
  tempx1[2][i] = 1
  tempx2[2][i] = 1

for i in randkp:
  tempx1[0][i] = x1[0][i]
  tempx1[1][i] = x1[1][i]
  tempx1[2][i] = x1[2][i]
  tempx2[0][i] = x2[0][i]
  tempx2[1][i] = x2[1][i]
  tempx2[2][i] = x2[2][i]

#### Now, we will apply the **3D Reconstruction** algorithm 

This has been applied as taught in class.

The following cell contains the function for the sum of the squares of the distances from the Origin $\bigg[\equiv \begin{pmatrix}  0\\ 0 \\ 1\end{pmatrix}\bigg]$ for the parameterized epipolar lines. This is also essentially the cost function:
\begin{equation}
\phantom{.}C(t) = d^{2} + d'^{2} = \frac{t^{2}}{1+t^{2}f^{2}} + \frac{t'^{2}}{1+t'^{2}f'^{2}}.
\end{equation}
We need the parameter value $t$ for which the cost function $C(t)$ attains its minimum value. We thus differentiate $C(t)$ and equate to $0$, to obtain:
\begin{equation}
\phantom{,}g(t) = t((at+b)^{2} + (f')^{2}(ct+d)^{2})^{2} + (1+t^{2}f^{2})^{2}(ct+d)(at+b)(bc-ad) = 0,
\end{equation}
where the transformed epipoles (obtained after translation and rotation) are denoted by $e_{1} = \begin{pmatrix} 1 \\ 0 \\ f\end{pmatrix}$ and $e_{2} = \begin{pmatrix} 1 \\ 0 \\ f'\end{pmatrix}$. \
We use *numpy.roots()* method to obtain the roots of $g(t)$. Then we check for which value of the roots thus obtained, $C(t)$ attains its minimum value. We retain that root, say $t^{*}$ to obtain:
\begin{equation}
\phantom{,}\hat{x} = \begin{pmatrix}\frac{t^{*2}f}{1+t^{*2}f^{2}} \\ \frac{t}{1+t^{*2}f^{2}} \\ 1 \end{pmatrix},
\end{equation} 
and
\begin{equation}
\phantom{,}\hat{x'} = \begin{pmatrix}\frac{t^{*'2}f'}{1+t^{*'2}f'^{2}} \\ \frac{t^{*'}}{1+t^{*'2}f'^{2}} \\ 1 \end{pmatrix},
\end{equation} 
where $t^{*'} = - \big(\frac{ct*+d}{at*+b}\big)$. These image points $\hat{x}$ and $\hat{x'}$ need to be inverse transformed through the respective matrices ($R^{T}$ and $T^{-1}$ for $\hat{x}$, and $R'^{T}$ and $T'^{-1}$ for $\hat{x'}$). Then, the *triangulate()* function should be called to estimate the world point $\hat{X}$. [Note that $R^{T} = R^{-1}$ and $R'^{T} = R'^{-1}$ since both of them are orthonormal rotation matrices.]

In [55]:
### The following is code for obtaining c(t)
## Note that g(t) is the first derivative of c(t)
## Setting g(t) = 0 obtain roots
## For one of the roots, the value of c(t) will be least
## Retain that t
def c_t(t, ax, bx, cx, dx, fx, gx):
  chi1 = (t**2)/(1+(t**2)*(fx**2))
  chi2 = ((cx*t+dx)**2)/((ax*t+bx)**2 + (gx**2)*((cx*t+dx)**2))
  return (chi1+chi2)

The following function is for **Reconstruction using Triangulation**. It takes image points $x$ and $x'$ (in camera planes corresponding to $P$ and $P'$; the 2 image points maybe the estimated ones obtained upon applying the 3D reconstruction algorithm), and computes the corresponding world point $X$ in homogeneous form (returned as *xest* in the function definition). \
Say we take $x = \begin{pmatrix} a \\ b\\ 1\end{pmatrix}$ and $x' = \begin{pmatrix}a' \\ b'\\1 \end{pmatrix}$, then $\hat{X}$ is estimated as:
\begin{equation}
\phantom{.}\begin{pmatrix}a \pi_{3}^{T} - \pi_{1}^{T} \\ b \pi_{3}^{T} - \pi_{2}^{T}\\ a' \pi_{3}^{'T} - \pi_{1}^{'T} \\ b' \pi_{3}^{'T} - \pi_{2}^{'T}
\end{pmatrix}\hat{X} = \begin{pmatrix}0 \\ 0 \\ 0 \\ 0\end{pmatrix}.
\end{equation}
Now, we again apply **SVD** technique to get $\hat{X}$.

In [56]:
def triangulate(x1_, x2_, P1_, P2_):
  row1 = x1_[0][0]*P1_[2, :] - P1_[0, :]
  row2 = x1_[1][0]*P1_[2, :] - P1_[1, :]
  row3 = x2_[0][0]*P2_[2, :] - P2_[0, :]
  row4 = x2_[1][0]*P2_[2, :] - P2_[1, :]
  evalmat = np.array([row1, row2, row3, row4])
  tmpest = dlt(evalmat)[:, -1].reshape((4, 1))
  xest = tmpest/tmpest[3][0]
  return xest

The following cell implements the **entire 3D reconstruction algorithm** for the 80 points to which noise has been added. The different steps have been clearly mentioned in the cell below, and are according to the notes provided in class. \
We are printing the estimated world point $\hat{X}$, followed by the original world point $X$. We also mention the indices for these points, out of the 100 world points. Then at the very end, we print the computed MSE with respect to the 80 world points. \
Let the set of all 100 indices be denoted by $N$ (which is the same as the notation $[n=100]$). Let $S$ be the subset of 20 random points chosen to estimate the Fundamental matrix $F$. Then the MSE is given by:
\begin{equation}
\phantom{,}MSE = \frac{1}{\mathrm{card}(N-S)} \sum_{i \in N-S} \|X_{arr}(i) - \hat{X}_{arr}(i)\|_{2}^{2},
\end{equation}
where $X_{arr}$ and $\hat{X}_{arr}$ represent the ensembles of the original and estimated world points respectively. Note that $\mathrm{card}(N-S) = 100 - 20 = 80$.

In [57]:
Xmse = 0
Xest_arr = np.zeros((4, 100))

for i in randkp:
  Xest_arr[0][i] = X[0][i]
  Xest_arr[1][i] = X[1][i]
  Xest_arr[2][i] = X[2][i]
  Xest_arr[3][i] = X[3][i]

for i in rem80:
  a1 = tempx1[0][i]
  b1 = tempx1[1][i]
  a2 = tempx2[0][i]
  b2 = tempx2[1][i]

  ### Step-1
  ## T1 is Tdash_inv_trans, T2 is T_inv
  T2 = np.array([[1, 0, 0], [0, 1, 0], [a2, b2, 1]])
  T1 = np.array([[1, 0, a1], [0, 1, b1], [0, 0, 1]])
  F1 = np.matmul(np.matmul(T2, F), T1)
  
  ### Step-2
  ## Obtaining left and right null vectors of Fnew

  e1 = dlt(F1)[:, -1].reshape((3, 1))
  e2 = dlt(np.transpose(F1))[:, -1].reshape((3, 1))
  kup1 = math.sqrt(e1[0, 0]**2 + e1[1, 0]**2)
  kup2 = math.sqrt(e2[0, 0]**2 + e2[1, 0]**2)
  e1[0, 0] = e1[0, 0]/kup1
  e1[1, 0] = e1[1, 0]/kup1
  e1[2, 0] = e1[2, 0]/kup1
  e2[0, 0] = e2[0, 0]/kup2
  e2[1, 0] = e2[1, 0]/kup2
  e2[2, 0] = e2[2, 0]/kup2
  
  ### Step-3
  ## Construct R1 and R2 to be multiplied with Fnew

  R1 = np.array([[e1[0, 0], e1[1, 0], 0], [-1*e1[1, 0], e1[0, 0], 0], [0, 0, 1]])
  R2 = np.array([[e2[0, 0], e2[1, 0], 0], [-1*e2[1, 0], e2[0, 0], 0], [0, 0, 1]])
  F2 = np.matmul(np.matmul(R2, F1), np.transpose(R1))
  e1 = np.matmul(R1, e1)
  e2 = np.matmul(R2, e2)
  ### Step-4: Obtaining optimum value of some root t
  ### Extract a, b, c, dfrom the new fundamental matrix F2
  ## F2 = [[dff' -cf' -df']
  ##       [-bf   a    b  ]
  ##       [-df   c    d  ]]

  a = F2[1, 1]
  b = F2[1, 2]
  c = F2[2, 1]
  d = F2[2, 2]
  f = e1[2, 0]
  g = e2[2, 0]

  # here we are building the coefficient array for computation of the roots
  # this coefficient array contains the coefficients in descending order of exponent
  # corresponding to the expansion of g(t) in terms of variable t
  coeff = np.array([0.0 for guy in range(7)])
  coeff[0] = a*b*(c**2)*(f**4) - (a**2)*c*(f**4)*d
  coeff[1] = a**4 + (c**4)*(g**4) + 2*(g**2)*(c**2)*(a**2) - (a**2)*(f**4)*(d**2) + (b**2)*(c**2)*(f**4)
  coeff[2] = 4*(a**3)*b + 4*a*b*(g**2)*(c**2) + 4*(g**2)*(a**2)*c*d + 4*(c**3)*d*(g**4) + (b**2)*c*(f**4)*d - a*b*(f**4)*(d**2) + 2*a*b*(c**2)*(f**2) - 2*(a**2)*c*(f**2)*d
  coeff[3] = 6*(a**2)*(b**2) + 2*(g**2)*(c**2)*(b**2) + 8*a*b*(g**2)*c*d + 2*(g**2)*(d**2)*(a**2) + 6*(c**2)*(d**2)*(g**4) - 2*(a**2)*(f**2)*(d**2) + 2*(b**2)*(c**2)*(f**2)
  coeff[4] = 4*a*(b**3) + 4*(g**2)*(b**2)*c*d + 4*a*b*(g**2)*(d**2) + 4*c*(d**3)*(g**4) + a*b*(c**2) - (a**2)*c*d + 2*(b**2)*c*(f**2)*d - 2*a*b*(f**2)*(d**2)
  coeff[5] = b**4 + 2*(g**2)*(d**2)*(b**2) + (d**4)*(g**4) - (a**2)*(d**2) + (b**2)*(c**2)
  coeff[6] = (b**2)*c*d - a*b*(d**2)
  
  valt = np.roots(coeff)
  
  minc = 1e30
  optroot = -1e-10
  for troot in valt:
    if(troot.imag <= 1e-6):
      cval = c_t(troot.real, a, b, c, d, f, g)
      if(cval < minc):
        minc = cval
        optroot = troot.real
  x1opt = np.zeros((3, 1))
  x2opt = np.zeros((3, 1))
  
  x1opt[0][0] = (optroot**2)*f
  x1opt[1][0] = optroot
  x1opt[2][0] = 1+(optroot**2)*(f**2)
  
  optdash = -1*(c*optroot+d)/(a*optroot+b)
  
  x2opt[0][0] = (optdash**2)*g
  x2opt[1][0] = optdash
  x2opt[2][0] = 1+(optdash**2)*(g**2)

  ### Step - 5
  ## Inverse rotate and translate to original system
  x1opt = np.matmul(np.transpose(R1), x1opt)
  x2opt = np.matmul(np.transpose(R2), x2opt)
  x1opt = np.matmul(np.array([[1, 0, a1], [0, 1, b1], [0, 0, 1]]), x1opt)
  x2opt = np.matmul(np.array([[1, 0, a2], [0, 1, b2], [0, 0, 1]]), x2opt)
  x1opt = x1opt/x1opt[2][0]
  x2opt = x2opt/x2opt[2][0]

  ### Step - 6
  ## Now let's bring in the old camera matrices P1 and P2
  ## x1opt, x2opt and the new to-be-estimated world point X are perfectly triangulated
  ## Let's estimate this world point X using triangulate() function defined earlier
  
  X_est = triangulate(x1opt, x2opt, P1, P2)

  Xest_arr[0][i] = X_est[0][0]
  Xest_arr[1][i] = X_est[1][0]
  Xest_arr[2][i] = X_est[2][0]
  Xest_arr[3][i] = X_est[3][0]

  print("Estimated X[",str(i), ']\t', "True X[", str(i), ']')
  print(X_est[0][0], '\t', X[0][i])
  print(X_est[1][0], '\t', X[1][i])
  print(X_est[2][0], '\t', X[2][i])
  print(X_est[3][0], '\t\t\t', X[3][i])
  print('\n')

  Xmse = Xmse + ((X_est[0][0]-X[0][i])**2 + (X_est[1][0]-X[1][i])**2 + (X_est[2][0]-X[2][i])**2)

Xmse = Xmse/(100-kp)
print("Final MSE for ", str(100-kp), " world points (estimated vs original) = ", Xmse)

Estimated X[ 0 ]	 True X[ 0 ]
-0.507267023132935 	 -0.170073485186932
-1.2020798628969385 	 -0.6493039473932762
2.2325984773413223 	 1.5464861826910614
1.0 			 1.0


Estimated X[ 1 ]	 True X[ 1 ]
0.3508255906796542 	 0.3793897829643355
-1.069252708259968 	 -1.137760074764976
0.4316416788835336 	 0.4743475195494361
1.0 			 1.0


Estimated X[ 2 ]	 True X[ 2 ]
-0.6502441944830456 	 -1.0051978366891274
0.6282529351469088 	 1.0182193745899766
-0.6612638611769742 	 -1.226429908547385
1.0 			 1.0


Estimated X[ 3 ]	 True X[ 3 ]
1.1640235773175056 	 1.6547040147659962
0.7378743388861148 	 1.1179225583854915
0.5170422732265176 	 0.65823665498504
1.0 			 1.0


Estimated X[ 4 ]	 True X[ 4 ]
0.6336922321846241 	 0.639801825435495
0.346734572372145 	 0.4176430601786701
-0.27938164333145427 	 -0.2600955246250126
1.0 			 1.0


Estimated X[ 5 ]	 True X[ 5 ]
1.4340267154215065 	 1.9646615110887728
0.19624556516477992 	 0.25378344574130035
-1.1995274751859704 	 -1.6544940354695095
1.0 			 1.0


Estimate